In [3]:
# Importanto as bibliotecas necessárias
import requests
import certifi
import pandas as pd
import time

import extrator
from io import BytesIO
import sys
import ast

df = pd.read_csv('candidatos.csv', dtype=object)

df['texto'] = ""
df['tamanho'] = ""
df['excec'] = ""
df['erro'] = ""

df2 = df
df = df.head(30)

In [4]:
start = time.perf_counter()

with requests.Session() as s:
    for i in range(len(df)):
        df.at[i,'tamanho'] = []
        df.at[i,'excec'] = []
        url_arqs = ast.literal_eval(df.loc[i, 'url_proposta'])
        texto_final = ""

        for j in url_arqs:
            r = s.get(j, verify=certifi.where(), timeout=500)
            t = r.content

            if sys.getsizeof(t) < 8000000:

                extracao = BytesIO(t)
                
                try:
                    texto_ext = extrator.convert_pdf_to_txt(extracao)
                    texto_ext = " ".join(texto_ext.split())
                    texto_final = f"{texto_final} {texto_ext}"
                    
                except:
                    df.at[i, 'erro'] = "#ERRO"
                    

            elif sys.getsizeof(t) > 8000000:
                df.at[i, 'tamanho'].append(j)

            else:
                df.at[i, 'excec'].append(j)

        df.at[i,'texto'] = texto_final
        
        print(f"#{i} {df.at[i, 'nm_candidato']} concluído")

        
finish = time.perf_counter()

print(f"tempo: {round(finish-start, 2)}")

df.to_csv('resultados.csv')

#0 FABIANO HENRIQUE DOS PASSOS concluído
#1 LEONILCO LIMA FEITOSA concluído
#2 TIAGO GOMES DIAS concluído
#3 ALEX CARNIEL concluído
#4 GILMAR DIAS DE OLIVEIRA concluído
#5 CLEIDSON RODRIGUES DE PAULA concluído
#6 LUIZ ANTONIO PEREIRA DE SOUZA concluído
#7 CEZAR OLIMPIO ZANDONA concluído
#8 OLIVAL SILVA DE OLIVEIRA concluído
#9 JUVENIL DOURADO DE SOUZA concluído
#10 WASHINGTON LUIS MOURA GALVÃO concluído
#11 JOSÉ DE FILIPPI JÚNIOR concluído
#12 ELISMAR BATISTA DE SOUSA concluído
#13 EDSOM LUIZ BAGETTI concluído
#14 ELISON RODRIGUES DE SOUZA concluído
#15 MAIK STÉFAN SANTOS MOREIRA concluído
#16 ELVIDIO PIRES DE SOUSA FILHO concluído
#17 MARCOS FABIANO DOS SANTOS TIBURCIO concluído
#18 AMAURI MAGNUS GERMANO concluído
#19 DORIVAL AMÂNCIO FROES concluído
#20 POMPILIO DE LOURDES CANAVEZ concluído
#21 JOÃO FRANCISCO DE SOUZA concluído
#22 MILTON INACIO DA SILVA concluído
#23 JOSE GERALDO ALVES DE ALMEIDA concluído
#24 AILTON PEREIRA GOULART concluído
#25 ANTONIO JOSE DE OLIVEIRA concluído
#2

NameError: name 'numero' is not defined

In [5]:
print(f"tempo: {round(finish-start, 2)}")

tempo: 19.81


-> primeiro pegar as ids dos 10
-> depois for loop para cada um
    -> Iniciar processo
    -> Extrair etc 

# MULTIPROCESSING

In [25]:
# Importanto as bibliotecas necessárias
import multiprocessing
from multiprocessing import process
import requests
import certifi
import pandas as pd

import extrator
from io import BytesIO
import sys
import ast

df = pd.read_csv('candidatos.csv', dtype=object)

df['texto'] = ""

processos = []

df = df.head(30)
df3 = df[0:0]

def extraido(numero):
    url_arqs = ast.literal_eval(df.loc[numero, 'url_proposta'])
    texto_final = ""

    for j in url_arqs:
        r = s.get(j, verify=certifi.where(), timeout=500)
        t = r.content

        if sys.getsizeof(t) < 8000000:

            extracao = BytesIO(t)

            try:
                texto_ext = extrator.convert_pdf_to_txt(extracao)
                texto_ext = " ".join(texto_ext.split())
                texto_final = f"{texto_final} {texto_ext}"

            except:
                texto_final = "#ERRO"

        elif sys.getsizeof(t) > 8000000:
            texto_final = "#TAMANHO"

        else:
            texto_final = "#EXCECAO"

    print(f"#{numero} {df.at[numero, 'nm_candidato']} concluído")

    df.at[numero,'texto'] = texto_final
    return df.iloc[numero]

with requests.Session() as s:
    for i in range(len(df)):
        p = multiprocessing.Process(target=extraido, args=(i,))
        p.start()
        

        processos.append(p)

for x in processos:
    x.join()


df.to_csv('resultados_teste.csv')

#6 LUIZ ANTONIO PEREIRA DE SOUZA concluído#18 AMAURI MAGNUS GERMANO concluído

#22 MILTON INACIO DA SILVA concluído
#12 ELISMAR BATISTA DE SOUSA concluído
#9 JUVENIL DOURADO DE SOUZA concluído
#13 EDSOM LUIZ BAGETTI concluído
#21 JOÃO FRANCISCO DE SOUZA concluído
#16 ELVIDIO PIRES DE SOUSA FILHO concluído
#19 DORIVAL AMÂNCIO FROES concluído
#26 ISAÍAS JOSÉ SILVA OLIVEIRA NETO concluído#5 CLEIDSON RODRIGUES DE PAULA concluído

#25 ANTONIO JOSE DE OLIVEIRA concluído
#17 MARCOS FABIANO DOS SANTOS TIBURCIO concluído
#7 CEZAR OLIMPIO ZANDONA concluído
#0 FABIANO HENRIQUE DOS PASSOS concluído
#8 OLIVAL SILVA DE OLIVEIRA concluído
#11 JOSÉ DE FILIPPI JÚNIOR concluído
#4 GILMAR DIAS DE OLIVEIRA concluído
#15 MAIK STÉFAN SANTOS MOREIRA concluído
#2 TIAGO GOMES DIAS concluído
#27 LUIZ GONZAGA ALVES DE LIMA FILHO concluído
#14 ELISON RODRIGUES DE SOUZA concluído
#20 POMPILIO DE LOURDES CANAVEZ concluído
#29 CARLOS ALBERTO LOPES DE OLIVEIRA concluído
#1 LEONILCO LIMA FEITOSA concluído
#10 WASHINGT

# OUTRA FORMA - A CORRETA -------------------------


In [13]:
# Importanto as bibliotecas necessárias
import concurrent.futures
import requests
import certifi
import pandas as pd
import time

import extrator
from io import BytesIO
import sys
import ast

df = pd.read_csv('candidatos.csv', dtype=object)

df['texto'] = ""

def extraido(numero):
    url_arqs = ast.literal_eval(df.loc[numero, 'url_proposta'])
    texto_final = ""

    for j in url_arqs:
        r = s.get(j, verify=certifi.where(), timeout=500)
        t = r.content

        if sys.getsizeof(t) < 8000000:

            extracao = BytesIO(t)

            try:
                texto_ext = extrator.convert_pdf_to_txt(extracao)
                texto_ext = " ".join(texto_ext.split())
                texto_final = f"{texto_final} {texto_ext}"

            except:
                return "#ERRO"

        elif sys.getsizeof(t) > 8000000:
            return "#TAMANHO"

        else:
            return "#EXCECAO"

    print(f"#{numero} {df.at[numero, 'nm_candidato']} concluído")
    
    dici_final = [numero,texto_final]

    return dici_final


start = time.perf_counter()

with requests.Session() as s:
    with concurrent.futures.ProcessPoolExecutor() as executor:
        texto_cache = [executor.submit(extraido, i) for i in range(len(df))]
        
        for f in texto_cache:
            f = f.result()
            df.at[f[0],'texto'] = f[1]
        


finish = time.perf_counter()

print(f"tempo: {round(finish-start, 2)}")

df.to_csv('resultado_final.csv')


#6 LUIZ ANTONIO PEREIRA DE SOUZA concluído
#0 FABIANO HENRIQUE DOS PASSOS concluído
#5 CLEIDSON RODRIGUES DE PAULA concluído
#7 CEZAR OLIMPIO ZANDONA concluído
#4 GILMAR DIAS DE OLIVEIRA concluído
#9 JUVENIL DOURADO DE SOUZA concluído
#8 OLIVAL SILVA DE OLIVEIRA concluído
#12 ELISMAR BATISTA DE SOUSA concluído
#2 TIAGO GOMES DIAS concluído
#13 EDSOM LUIZ BAGETTI concluído
#1 LEONILCO LIMA FEITOSA concluído
#11 JOSÉ DE FILIPPI JÚNIOR concluído
#3 ALEX CARNIEL concluído
#16 ELVIDIO PIRES DE SOUSA FILHO concluído
#18 AMAURI MAGNUS GERMANO concluído
#22 MILTON INACIO DA SILVA concluído
#17 MARCOS FABIANO DOS SANTOS TIBURCIO concluído
#21 JOÃO FRANCISCO DE SOUZA concluído
#15 MAIK STÉFAN SANTOS MOREIRA concluído
#19 DORIVAL AMÂNCIO FROES concluído
#10 WASHINGTON LUIS MOURA GALVÃO concluído
#14 ELISON RODRIGUES DE SOUZA concluído
#25 ANTONIO JOSE DE OLIVEIRA concluído
#26 ISAÍAS JOSÉ SILVA OLIVEIRA NETO concluído
#31 GIL TAVARES concluído
#20 POMPILIO DE LOURDES CANAVEZ concluído
#33 ANTÔNIO

Process ForkProcess-28:
Process ForkProcess-30:
Process ForkProcess-27:
Process ForkProcess-25:
Process ForkProcess-31:
Process ForkProcess-26:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/mul

KeyboardInterrupt: 

# Corrigindo os que faltaram

In [ ]:
# Importanto as bibliotecas necessárias
import concurrent.futures
import requests
import certifi
import pandas as pd
import numpy as np
import time

import extrator
from io import BytesIO
import sys
import ast


df4 = pd.read_csv('resultado_final2.csv', dtype=object)

def extraido(numero):
    url_arqs = ast.literal_eval(df4.loc[numero, 'url_proposta'])
    texto_final = ""

    for j in url_arqs:
        r = s.get(j, verify=certifi.where(), timeout=500)
        t = r.content

        if sys.getsizeof(t) < 8000000:

            extracao = BytesIO(t)

            try:
                texto_ext = extrator.convert_pdf_to_txt(extracao)
                texto_ext = " ".join(texto_ext.split())
                texto_final = f"{texto_final} {texto_ext}"

            except:
                return "#ERRO"

        elif sys.getsizeof(t) > 8000000:
            return "#TAMANHO"

        else:
            return "#EXCECAO"

    print(f"#{numero} {df4.at[numero, 'nm_candidato']} concluído")
    
    dici_final = [numero,texto_final]

    return dici_final


start = time.perf_counter()

with requests.Session() as s:
    with concurrent.futures.ProcessPoolExecutor() as executor:
        lista = []
        
        for linha in range(len(df4)):
            if type(df4.loc[linha, 'texto']) == float :
                lista.append(linha)
                print(f"Vazio: #{linha} {df4.loc[linha, 'nm_candidato']}")
                
        texto_cache = [executor.submit(extraido, i) for i in lista]
        
        for f in texto_cache:
            f = f.result()
            df4.at[f[0],'texto'] = f[1]
        


finish = time.perf_counter()

print(f"tempo: {round(finish-start, 2)}")

df4.to_csv('final_final.csv')


Vazio: #34 JONAS CARRILHO ROSA
Vazio: #35 VALDIR MEDEIROS MAXIMINO
Vazio: #36 ODAIR OLEGARIO DE LIMA
Vazio: #53 ANTONIO DE FREITAS
Vazio: #58 VANDERSON VIRGÍLIO CAMPOS DOS SANTOS
Vazio: #74 ALEXANDRE FERNANDES HABIBE
Vazio: #83 MARIA BETÂNIA SOUZA FÉLIX
Vazio: #190 ÁUREO BEZERRA GOMES
Vazio: #209 PEDRO IVO BUENO
Vazio: #223 ONOFRE DE SOUZA
Vazio: #230 CARLOS ALBERTO ANDRADE OLIVEIRA
Vazio: #238 JOSE OLIVEIRA LEITE
Vazio: #254 LEONARDO FARIA ZAMPA
Vazio: #264 PEDRO HENRIQUE SOARES BRAGA
Vazio: #291 JANDIRA SAMPAIO CAVICHINI GUTIERREZ
Vazio: #298 MICHELA DA SILVA FREITAS
Vazio: #307 JOAO BATISTA REUS VIEIRA
Vazio: #319 CLÓVIS FREIBERGER JUNIOR
Vazio: #328 GILSON ALVES RODRIGUES
Vazio: #336 MOACIR MARTINS DA COSTA JUNIOR
Vazio: #344 EDIMILSON OLIVEIRA DA SILVA
Vazio: #349 MOISES MESSIAS LIMA
Vazio: #370 NELENE DA COSTA GOMES
Vazio: #388 JOAO ANTONIO KLEINUBING
Vazio: #396 IDALINA GONCALVES MAGALHAES
Vazio: #475 ANTÔNIO EUSTÁQUIO DIAS
Vazio: #573 WILLIAM FERNANDES MUSSI
Vazio: #585 AROLDO 

ai
